In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 64
tree_depth = 12
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [3]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.212848663330078 | KNN Loss: 6.229985237121582 | BCE Loss: 1.9828636646270752
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.168519973754883 | KNN Loss: 6.229908466339111 | BCE Loss: 1.9386110305786133
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.14419937133789 | KNN Loss: 6.229614734649658 | BCE Loss: 1.9145846366882324
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.14875316619873 | KNN Loss: 6.229379653930664 | BCE Loss: 1.919373631477356
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.138741493225098 | KNN Loss: 6.229327201843262 | BCE Loss: 1.9094146490097046
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.163969039916992 | KNN Loss: 6.229116439819336 | BCE Loss: 1.9348526000976562
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.142416000366211 | KNN Loss: 6.228728771209717 | BCE Loss: 1.9136877059936523
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.133009910583496 | KNN Loss: 6.228733062744141 | BCE Loss: 1.90427720

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 7.1136908531188965 | KNN Loss: 6.006924152374268 | BCE Loss: 1.106766700744629
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 7.062594890594482 | KNN Loss: 5.958533763885498 | BCE Loss: 1.1040611267089844
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 7.014702796936035 | KNN Loss: 5.912962436676025 | BCE Loss: 1.1017404794692993
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 6.985175132751465 | KNN Loss: 5.881042003631592 | BCE Loss: 1.104133129119873
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 6.982003211975098 | KNN Loss: 5.843730926513672 | BCE Loss: 1.1382720470428467
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 6.8553266525268555 | KNN Loss: 5.777772426605225 | BCE Loss: 1.0775541067123413
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 6.823657989501953 | KNN Loss: 5.716406345367432 | BCE Loss: 1.1072516441345215
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 6.77890157699585 | KNN Loss: 5.681288242340088 | BCE Loss:

Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 5.507378101348877 | KNN Loss: 4.423064231872559 | BCE Loss: 1.084313988685608
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 5.532404899597168 | KNN Loss: 4.467677116394043 | BCE Loss: 1.064727544784546
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 5.539498329162598 | KNN Loss: 4.463871002197266 | BCE Loss: 1.0756272077560425
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 5.464534759521484 | KNN Loss: 4.4394354820251465 | BCE Loss: 1.0250990390777588
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 5.4879913330078125 | KNN Loss: 4.418454170227051 | BCE Loss: 1.0695372819900513
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 5.512447357177734 | KNN Loss: 4.455063343048096 | BCE Loss: 1.0573842525482178
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 5.4826130867004395 | KNN Loss: 4.447882175445557 | BCE Loss: 1.0347310304641724
Epoch 22 / 500 | iteration 25 / 30 | Total Loss: 5.572408199310303 | KNN Loss: 4.486761569976807 | BCE Lo

Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 5.467789649963379 | KNN Loss: 4.427908420562744 | BCE Loss: 1.0398812294006348
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 5.515353679656982 | KNN Loss: 4.440191745758057 | BCE Loss: 1.0751619338989258
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 5.499302864074707 | KNN Loss: 4.43162727355957 | BCE Loss: 1.0676755905151367
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 5.4820556640625 | KNN Loss: 4.421136379241943 | BCE Loss: 1.060919165611267
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 5.483309268951416 | KNN Loss: 4.434974193572998 | BCE Loss: 1.048335075378418
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 5.489395618438721 | KNN Loss: 4.430936813354492 | BCE Loss: 1.058458924293518
Epoch 33 / 500 | iteration 15 / 30 | Total Loss: 5.49208402633667 | KNN Loss: 4.427916526794434 | BCE Loss: 1.0641676187515259
Epoch 33 / 500 | iteration 20 / 30 | Total Loss: 5.442554473876953 | KNN Loss: 4.402794361114502 | BCE Loss: 1.03

Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 5.453668594360352 | KNN Loss: 4.428226947784424 | BCE Loss: 1.0254415273666382
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 5.423710823059082 | KNN Loss: 4.404366493225098 | BCE Loss: 1.0193440914154053
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 5.475948333740234 | KNN Loss: 4.4180426597595215 | BCE Loss: 1.057905673980713
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 5.454903602600098 | KNN Loss: 4.396880626678467 | BCE Loss: 1.0580229759216309
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 5.440617561340332 | KNN Loss: 4.386813640594482 | BCE Loss: 1.05380380153656
Epoch 44 / 500 | iteration 5 / 30 | Total Loss: 5.4464497566223145 | KNN Loss: 4.408674716949463 | BCE Loss: 1.037774920463562
Epoch 44 / 500 | iteration 10 / 30 | Total Loss: 5.449211120605469 | KNN Loss: 4.405559062957764 | BCE Loss: 1.043652057647705
Epoch 44 / 500 | iteration 15 / 30 | Total Loss: 5.44399356842041 | KNN Loss: 4.415781497955322 | BCE Loss: 1

...

Epoch 449 / 500 | iteration 10 / 30 | Total Loss: 5.347927093505859 | KNN Loss: 4.343445301055908 | BCE Loss: 1.004481554031372
Epoch 449 / 500 | iteration 15 / 30 | Total Loss: 5.371289253234863 | KNN Loss: 4.334761142730713 | BCE Loss: 1.03652822971344
Epoch 449 / 500 | iteration 20 / 30 | Total Loss: 5.332276344299316 | KNN Loss: 4.34283447265625 | BCE Loss: 0.9894418120384216
Epoch 449 / 500 | iteration 25 / 30 | Total Loss: 5.342615127563477 | KNN Loss: 4.345034599304199 | BCE Loss: 0.9975806474685669
Epoch 450 / 500 | iteration 0 / 30 | Total Loss: 5.359007835388184 | KNN Loss: 4.367350101470947 | BCE Loss: 0.9916574954986572
Epoch 450 / 500 | iteration 5 / 30 | Total Loss: 5.344439506530762 | KNN Loss: 4.330578327178955 | BCE Loss: 1.0138612985610962
Epoch 450 / 500 | iteration 10 / 30 | Total Loss: 5.396506309509277 | KNN Loss: 4.3855133056640625 | BCE Loss: 1.0109928846359253
Epoch 450 / 500 | iteration 15 / 30 | Total Loss: 5.382510662078857 | KNN Loss: 4.359109878540039 | BC

Epoch 460 / 500 | iteration 0 / 30 | Total Loss: 5.353022575378418 | KNN Loss: 4.3539886474609375 | BCE Loss: 0.9990341663360596
Epoch 460 / 500 | iteration 5 / 30 | Total Loss: 5.396484375 | KNN Loss: 4.3535637855529785 | BCE Loss: 1.0429205894470215
Epoch 460 / 500 | iteration 10 / 30 | Total Loss: 5.351693630218506 | KNN Loss: 4.327581405639648 | BCE Loss: 1.0241122245788574
Epoch 460 / 500 | iteration 15 / 30 | Total Loss: 5.358039379119873 | KNN Loss: 4.322794437408447 | BCE Loss: 1.0352450609207153
Epoch 460 / 500 | iteration 20 / 30 | Total Loss: 5.373109817504883 | KNN Loss: 4.332299709320068 | BCE Loss: 1.0408098697662354
Epoch 460 / 500 | iteration 25 / 30 | Total Loss: 5.381913185119629 | KNN Loss: 4.349060535430908 | BCE Loss: 1.0328526496887207
Epoch 461 / 500 | iteration 0 / 30 | Total Loss: 5.328824043273926 | KNN Loss: 4.314085960388184 | BCE Loss: 1.0147383213043213
Epoch 461 / 500 | iteration 5 / 30 | Total Loss: 5.407964706420898 | KNN Loss: 4.3662428855896 | BCE Los

Epoch 470 / 500 | iteration 20 / 30 | Total Loss: 5.346129894256592 | KNN Loss: 4.314916133880615 | BCE Loss: 1.0312138795852661
Epoch 470 / 500 | iteration 25 / 30 | Total Loss: 5.4247517585754395 | KNN Loss: 4.376527309417725 | BCE Loss: 1.0482245683670044
Epoch 471 / 500 | iteration 0 / 30 | Total Loss: 5.441575050354004 | KNN Loss: 4.41087007522583 | BCE Loss: 1.0307047367095947
Epoch 471 / 500 | iteration 5 / 30 | Total Loss: 5.354607582092285 | KNN Loss: 4.321402072906494 | BCE Loss: 1.0332057476043701
Epoch 471 / 500 | iteration 10 / 30 | Total Loss: 5.36297607421875 | KNN Loss: 4.337094783782959 | BCE Loss: 1.025881052017212
Epoch 471 / 500 | iteration 15 / 30 | Total Loss: 5.333868980407715 | KNN Loss: 4.327200889587402 | BCE Loss: 1.0066680908203125
Epoch 471 / 500 | iteration 20 / 30 | Total Loss: 5.383738040924072 | KNN Loss: 4.366393566131592 | BCE Loss: 1.017344355583191
Epoch 471 / 500 | iteration 25 / 30 | Total Loss: 5.364250659942627 | KNN Loss: 4.344139575958252 | BC

Epoch 481 / 500 | iteration 10 / 30 | Total Loss: 5.380999565124512 | KNN Loss: 4.382349491119385 | BCE Loss: 0.9986500144004822
Epoch 481 / 500 | iteration 15 / 30 | Total Loss: 5.323890209197998 | KNN Loss: 4.32285737991333 | BCE Loss: 1.001032829284668
Epoch 481 / 500 | iteration 20 / 30 | Total Loss: 5.402249336242676 | KNN Loss: 4.356246471405029 | BCE Loss: 1.0460028648376465
Epoch 481 / 500 | iteration 25 / 30 | Total Loss: 5.36060905456543 | KNN Loss: 4.328298568725586 | BCE Loss: 1.0323104858398438
Epoch 482 / 500 | iteration 0 / 30 | Total Loss: 5.40244722366333 | KNN Loss: 4.394598484039307 | BCE Loss: 1.0078487396240234
Epoch 482 / 500 | iteration 5 / 30 | Total Loss: 5.389484405517578 | KNN Loss: 4.368040561676025 | BCE Loss: 1.0214439630508423
Epoch 482 / 500 | iteration 10 / 30 | Total Loss: 5.365406036376953 | KNN Loss: 4.366781711578369 | BCE Loss: 0.9986240863800049
Epoch 482 / 500 | iteration 15 / 30 | Total Loss: 5.40535306930542 | KNN Loss: 4.355584144592285 | BCE 

Epoch   492: reducing learning rate of group 0 to 5.5221e-08.
Epoch 492 / 500 | iteration 0 / 30 | Total Loss: 5.350766658782959 | KNN Loss: 4.337794780731201 | BCE Loss: 1.0129719972610474
Epoch 492 / 500 | iteration 5 / 30 | Total Loss: 5.3958539962768555 | KNN Loss: 4.357967376708984 | BCE Loss: 1.0378867387771606
Epoch 492 / 500 | iteration 10 / 30 | Total Loss: 5.367641448974609 | KNN Loss: 4.3413214683532715 | BCE Loss: 1.026320219039917
Epoch 492 / 500 | iteration 15 / 30 | Total Loss: 5.3972930908203125 | KNN Loss: 4.384162902832031 | BCE Loss: 1.0131301879882812
Epoch 492 / 500 | iteration 20 / 30 | Total Loss: 5.36600399017334 | KNN Loss: 4.376267910003662 | BCE Loss: 0.9897359609603882
Epoch 492 / 500 | iteration 25 / 30 | Total Loss: 5.402640342712402 | KNN Loss: 4.371490955352783 | BCE Loss: 1.03114914894104
Epoch 493 / 500 | iteration 0 / 30 | Total Loss: 5.321385383605957 | KNN Loss: 4.327150344848633 | BCE Loss: 0.9942350387573242
Epoch 493 / 500 | iteration 5 / 30 | To

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 1.8325,  4.6160,  3.0677,  2.2880,  4.1206,  0.6368,  2.1909,  1.6315,
          1.8769,  2.4564,  2.1214,  1.3905,  1.1680,  2.0682,  1.4719,  0.8914,
          2.3867,  1.8554,  3.2410,  2.5316,  2.0258,  1.7814,  2.7396,  2.8656,
          1.6242,  1.8446,  1.2237,  0.8096,  1.0281,  0.5324,  0.1516,  0.8719,
         -0.3351,  0.9580,  2.0002,  0.9887,  0.4817,  3.2517,  0.8580,  0.8546,
          0.7083, -1.2026, -0.3733,  2.2330,  2.6924,  0.8294,  0.1402, -0.0804,
          1.7870,  1.6762,  2.2953,  0.1201,  0.7758,  0.4457, -0.4955,  1.0042,
          1.3151,  1.6255,  1.2598,  1.7012,  0.4507,  0.7257,  0.1015,  1.6082,
          1.2640,  1.6263, -1.6217,  0.1696,  2.7663,  1.7402,  2.9492,  0.2148,
          1.1837,  2.8738,  1.5330,  1.7702,  0.6192,  0.5534, -0.3117,  1.0783,
         -0.0685,  0.2990,  2.1036, -0.3987,  0.5665, -1.0399, -2.3851,  0.2871,
          0.5592, -1.7341,  0.2284, -0.0333, -0.1515, -1.0996,  0.6627,  0.8139,
         -0.4803, -0.3365,  

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [10]:
dataset_ = [d[0].cpu() for d in dataset]

In [11]:
model = model.eval().cpu()
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 83.33it/s]


In [12]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [13]:
clusters = DBSCAN(eps=0.2, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [14]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [16]:
tensor_dataset = torch.stack(dataset_)

In [17]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [18]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [19]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [20]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [21]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [22]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [23]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [24]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [25]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [26]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [27]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [28]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [29]:
losses = []
accs = []
sparsity = []

In [30]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
layer 7: 0.0
layer 8: 0.0
layer 9: 0.0
layer 10: 0.0
Epoch: 00 | Batch: 000 / 028 | Total loss: 9.634 | Reg loss: 0.014 | Tree loss: 9.634 | Accuracy: 0.000000 | 3.626 sec/iter
Epoch: 00 | Batch: 001 / 028 | Total loss: 9.632 | Reg loss: 0.013 | Tree loss: 9.632 | Accuracy: 0.000000 | 3.556 sec/iter
Epoch: 00 | Batch: 002 / 028 | Total loss: 9.629 | Reg loss: 0.012 | Tree loss: 9.629 | Accuracy: 0.000000 | 3.564 sec/iter
Epoch: 00 | Batch: 003 / 028 | Total loss: 9.625 | Reg loss: 0.011 | Tree loss: 9.625 | Accuracy: 0.000000 | 4.062 sec/iter
Epoch: 00 | Batch: 004 / 028 | Total loss: 9.624 | Reg loss: 0.010 | Tree loss: 9.624 | Accuracy: 0.000000 | 4.377 sec/iter
Epoch: 00 | Batch: 005 / 028 | Total loss: 9.621 | Reg loss: 0.009 | Tree loss: 9.621 | Accuracy: 0.000000 | 4.601 sec/iter
Epoch: 00 | Batch: 006 / 028 | Total loss: 9.618 | Reg loss: 0.008 | Tree loss: 9.618 | 

Epoch: 02 | Batch: 004 / 028 | Total loss: 9.558 | Reg loss: 0.007 | Tree loss: 9.558 | Accuracy: 0.171875 | 6.834 sec/iter
Epoch: 02 | Batch: 005 / 028 | Total loss: 9.555 | Reg loss: 0.007 | Tree loss: 9.555 | Accuracy: 0.199219 | 6.833 sec/iter
Epoch: 02 | Batch: 006 / 028 | Total loss: 9.556 | Reg loss: 0.007 | Tree loss: 9.556 | Accuracy: 0.169922 | 6.83 sec/iter
Epoch: 02 | Batch: 007 / 028 | Total loss: 9.552 | Reg loss: 0.007 | Tree loss: 9.552 | Accuracy: 0.179688 | 6.828 sec/iter
Epoch: 02 | Batch: 008 / 028 | Total loss: 9.551 | Reg loss: 0.007 | Tree loss: 9.551 | Accuracy: 0.175781 | 6.828 sec/iter
Epoch: 02 | Batch: 009 / 028 | Total loss: 9.546 | Reg loss: 0.008 | Tree loss: 9.546 | Accuracy: 0.228516 | 6.826 sec/iter
Epoch: 02 | Batch: 010 / 028 | Total loss: 9.544 | Reg loss: 0.008 | Tree loss: 9.544 | Accuracy: 0.220703 | 6.827 sec/iter
Epoch: 02 | Batch: 011 / 028 | Total loss: 9.540 | Reg loss: 0.008 | Tree loss: 9.540 | Accuracy: 0.232422 | 6.825 sec/iter
Epoch: 02

Epoch: 04 | Batch: 009 / 028 | Total loss: 9.444 | Reg loss: 0.013 | Tree loss: 9.444 | Accuracy: 0.222656 | 6.774 sec/iter
Epoch: 04 | Batch: 010 / 028 | Total loss: 9.435 | Reg loss: 0.013 | Tree loss: 9.435 | Accuracy: 0.205078 | 6.777 sec/iter
Epoch: 04 | Batch: 011 / 028 | Total loss: 9.427 | Reg loss: 0.014 | Tree loss: 9.427 | Accuracy: 0.177734 | 6.78 sec/iter
Epoch: 04 | Batch: 012 / 028 | Total loss: 9.421 | Reg loss: 0.014 | Tree loss: 9.421 | Accuracy: 0.203125 | 6.783 sec/iter
Epoch: 04 | Batch: 013 / 028 | Total loss: 9.416 | Reg loss: 0.015 | Tree loss: 9.416 | Accuracy: 0.177734 | 6.786 sec/iter
Epoch: 04 | Batch: 014 / 028 | Total loss: 9.403 | Reg loss: 0.015 | Tree loss: 9.403 | Accuracy: 0.205078 | 6.789 sec/iter
Epoch: 04 | Batch: 015 / 028 | Total loss: 9.400 | Reg loss: 0.015 | Tree loss: 9.400 | Accuracy: 0.183594 | 6.791 sec/iter
Epoch: 04 | Batch: 016 / 028 | Total loss: 9.377 | Reg loss: 0.016 | Tree loss: 9.377 | Accuracy: 0.169922 | 6.795 sec/iter
Epoch: 04

Epoch: 06 | Batch: 014 / 028 | Total loss: 9.040 | Reg loss: 0.021 | Tree loss: 9.040 | Accuracy: 0.156250 | 6.823 sec/iter
Epoch: 06 | Batch: 015 / 028 | Total loss: 9.006 | Reg loss: 0.021 | Tree loss: 9.006 | Accuracy: 0.162109 | 6.823 sec/iter
Epoch: 06 | Batch: 016 / 028 | Total loss: 8.982 | Reg loss: 0.022 | Tree loss: 8.982 | Accuracy: 0.210938 | 6.823 sec/iter
Epoch: 06 | Batch: 017 / 028 | Total loss: 8.968 | Reg loss: 0.022 | Tree loss: 8.968 | Accuracy: 0.171875 | 6.823 sec/iter
Epoch: 06 | Batch: 018 / 028 | Total loss: 8.960 | Reg loss: 0.023 | Tree loss: 8.960 | Accuracy: 0.169922 | 6.823 sec/iter
Epoch: 06 | Batch: 019 / 028 | Total loss: 8.906 | Reg loss: 0.023 | Tree loss: 8.906 | Accuracy: 0.167969 | 6.823 sec/iter
Epoch: 06 | Batch: 020 / 028 | Total loss: 8.885 | Reg loss: 0.024 | Tree loss: 8.885 | Accuracy: 0.195312 | 6.823 sec/iter
Epoch: 06 | Batch: 021 / 028 | Total loss: 8.881 | Reg loss: 0.024 | Tree loss: 8.881 | Accuracy: 0.208984 | 6.823 sec/iter
Epoch: 0

Epoch: 08 | Batch: 019 / 028 | Total loss: 8.379 | Reg loss: 0.026 | Tree loss: 8.379 | Accuracy: 0.195312 | 6.832 sec/iter
Epoch: 08 | Batch: 020 / 028 | Total loss: 8.336 | Reg loss: 0.027 | Tree loss: 8.336 | Accuracy: 0.201172 | 6.831 sec/iter
Epoch: 08 | Batch: 021 / 028 | Total loss: 8.329 | Reg loss: 0.027 | Tree loss: 8.329 | Accuracy: 0.173828 | 6.83 sec/iter
Epoch: 08 | Batch: 022 / 028 | Total loss: 8.321 | Reg loss: 0.027 | Tree loss: 8.321 | Accuracy: 0.179688 | 6.83 sec/iter
Epoch: 08 | Batch: 023 / 028 | Total loss: 8.288 | Reg loss: 0.028 | Tree loss: 8.288 | Accuracy: 0.181641 | 6.829 sec/iter
Epoch: 08 | Batch: 024 / 028 | Total loss: 8.234 | Reg loss: 0.028 | Tree loss: 8.234 | Accuracy: 0.179688 | 6.829 sec/iter
Epoch: 08 | Batch: 025 / 028 | Total loss: 8.216 | Reg loss: 0.028 | Tree loss: 8.216 | Accuracy: 0.175781 | 6.83 sec/iter
Epoch: 08 | Batch: 026 / 028 | Total loss: 8.206 | Reg loss: 0.029 | Tree loss: 8.206 | Accuracy: 0.197266 | 6.831 sec/iter
Epoch: 08 |

...

layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
layer 9: 0.9821428571428573
layer 10: 0.9821428571428573
Epoch: 91 | Batch: 000 / 028 | Total loss: 2.279 | Reg loss: 0.035 | Tree loss: 2.279 | Accuracy: 0.199219 | 7.021 sec/iter
Epoch: 91 | Batch: 001 / 028 | Total loss: 2.288 | Reg loss: 0.035 | Tree loss: 2.288 | Accuracy: 0.197266 | 7.02 sec/iter
Epoch: 91 | Batch: 002 / 028 | Total loss: 2.263 | Reg loss: 0.035 | Tree loss: 2.263 | Accuracy: 0.197266 | 7.02 sec/iter
Epoch: 91 | Batch: 003 / 028 | Total loss: 2.358 | Reg loss: 0.035 | Tree loss: 2.358 | Accuracy: 0.197266 | 7.019 sec/iter
Epoch: 91 | Batch: 004 / 028 | Total loss: 2.224 | Reg loss: 0.035 | Tree loss: 2.224 | Accuracy: 0.230469 | 7.019 sec/iter
Epoch: 91 | Batch: 005 / 028 | Total loss: 2.245 | Reg loss: 0.035 | Tree loss: 2.245 | Accuracy: 0.201172 | 7.019 sec/iter
Epoch: 91 | Batch: 006 / 028 | Total loss: 2.258 | Reg loss: 0.035 | Tree loss: 2.258 | Accuracy: 0.203125 | 7.019 sec

Epoch: 93 | Batch: 004 / 028 | Total loss: 2.242 | Reg loss: 0.034 | Tree loss: 2.242 | Accuracy: 0.193359 | 7.03 sec/iter
Epoch: 93 | Batch: 005 / 028 | Total loss: 2.227 | Reg loss: 0.034 | Tree loss: 2.227 | Accuracy: 0.187500 | 7.03 sec/iter
Epoch: 93 | Batch: 006 / 028 | Total loss: 2.293 | Reg loss: 0.034 | Tree loss: 2.293 | Accuracy: 0.226562 | 7.03 sec/iter
Epoch: 93 | Batch: 007 / 028 | Total loss: 2.326 | Reg loss: 0.034 | Tree loss: 2.326 | Accuracy: 0.177734 | 7.03 sec/iter
Epoch: 93 | Batch: 008 / 028 | Total loss: 2.276 | Reg loss: 0.034 | Tree loss: 2.276 | Accuracy: 0.187500 | 7.03 sec/iter
Epoch: 93 | Batch: 009 / 028 | Total loss: 2.234 | Reg loss: 0.034 | Tree loss: 2.234 | Accuracy: 0.207031 | 7.03 sec/iter
Epoch: 93 | Batch: 010 / 028 | Total loss: 2.216 | Reg loss: 0.034 | Tree loss: 2.216 | Accuracy: 0.195312 | 7.03 sec/iter
Epoch: 93 | Batch: 011 / 028 | Total loss: 2.255 | Reg loss: 0.034 | Tree loss: 2.255 | Accuracy: 0.158203 | 7.031 sec/iter
Epoch: 93 | Bat

Epoch: 95 | Batch: 009 / 028 | Total loss: 2.289 | Reg loss: 0.034 | Tree loss: 2.289 | Accuracy: 0.167969 | 7.039 sec/iter
Epoch: 95 | Batch: 010 / 028 | Total loss: 2.211 | Reg loss: 0.034 | Tree loss: 2.211 | Accuracy: 0.210938 | 7.038 sec/iter
Epoch: 95 | Batch: 011 / 028 | Total loss: 2.209 | Reg loss: 0.034 | Tree loss: 2.209 | Accuracy: 0.199219 | 7.038 sec/iter
Epoch: 95 | Batch: 012 / 028 | Total loss: 2.208 | Reg loss: 0.034 | Tree loss: 2.208 | Accuracy: 0.179688 | 7.038 sec/iter
Epoch: 95 | Batch: 013 / 028 | Total loss: 2.185 | Reg loss: 0.034 | Tree loss: 2.185 | Accuracy: 0.191406 | 7.038 sec/iter
Epoch: 95 | Batch: 014 / 028 | Total loss: 2.174 | Reg loss: 0.034 | Tree loss: 2.174 | Accuracy: 0.214844 | 7.038 sec/iter
Epoch: 95 | Batch: 015 / 028 | Total loss: 2.199 | Reg loss: 0.034 | Tree loss: 2.199 | Accuracy: 0.183594 | 7.038 sec/iter
Epoch: 95 | Batch: 016 / 028 | Total loss: 2.225 | Reg loss: 0.034 | Tree loss: 2.225 | Accuracy: 0.183594 | 7.038 sec/iter
Epoch: 9

Epoch: 97 | Batch: 014 / 028 | Total loss: 2.157 | Reg loss: 0.034 | Tree loss: 2.157 | Accuracy: 0.197266 | 7.047 sec/iter
Epoch: 97 | Batch: 015 / 028 | Total loss: 2.205 | Reg loss: 0.034 | Tree loss: 2.205 | Accuracy: 0.199219 | 7.047 sec/iter
Epoch: 97 | Batch: 016 / 028 | Total loss: 2.207 | Reg loss: 0.034 | Tree loss: 2.207 | Accuracy: 0.201172 | 7.047 sec/iter
Epoch: 97 | Batch: 017 / 028 | Total loss: 2.198 | Reg loss: 0.034 | Tree loss: 2.198 | Accuracy: 0.181641 | 7.047 sec/iter
Epoch: 97 | Batch: 018 / 028 | Total loss: 2.214 | Reg loss: 0.034 | Tree loss: 2.214 | Accuracy: 0.212891 | 7.047 sec/iter
Epoch: 97 | Batch: 019 / 028 | Total loss: 2.157 | Reg loss: 0.034 | Tree loss: 2.157 | Accuracy: 0.207031 | 7.047 sec/iter
Epoch: 97 | Batch: 020 / 028 | Total loss: 2.169 | Reg loss: 0.034 | Tree loss: 2.169 | Accuracy: 0.199219 | 7.047 sec/iter
Epoch: 97 | Batch: 021 / 028 | Total loss: 2.174 | Reg loss: 0.034 | Tree loss: 2.174 | Accuracy: 0.203125 | 7.047 sec/iter
Epoch: 9

Epoch: 99 | Batch: 019 / 028 | Total loss: 2.194 | Reg loss: 0.034 | Tree loss: 2.194 | Accuracy: 0.214844 | 7.047 sec/iter
Epoch: 99 | Batch: 020 / 028 | Total loss: 2.189 | Reg loss: 0.034 | Tree loss: 2.189 | Accuracy: 0.193359 | 7.047 sec/iter
Epoch: 99 | Batch: 021 / 028 | Total loss: 2.190 | Reg loss: 0.034 | Tree loss: 2.190 | Accuracy: 0.197266 | 7.047 sec/iter
Epoch: 99 | Batch: 022 / 028 | Total loss: 2.144 | Reg loss: 0.034 | Tree loss: 2.144 | Accuracy: 0.203125 | 7.047 sec/iter
Epoch: 99 | Batch: 023 / 028 | Total loss: 2.177 | Reg loss: 0.034 | Tree loss: 2.177 | Accuracy: 0.203125 | 7.047 sec/iter
Epoch: 99 | Batch: 024 / 028 | Total loss: 2.152 | Reg loss: 0.034 | Tree loss: 2.152 | Accuracy: 0.169922 | 7.048 sec/iter
Epoch: 99 | Batch: 025 / 028 | Total loss: 2.144 | Reg loss: 0.034 | Tree loss: 2.144 | Accuracy: 0.187500 | 7.048 sec/iter
Epoch: 99 | Batch: 026 / 028 | Total loss: 2.131 | Reg loss: 0.034 | Tree loss: 2.131 | Accuracy: 0.210938 | 7.048 sec/iter
Epoch: 9

In [31]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [33]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 11.93182416425375


# Extract Rules

# Accumulate samples in the leaves

In [34]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 3799


In [35]:
method = 'greedy'

In [36]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [37]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))


============== Pattern 1 ==============
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============
============== Pattern 25

============== Pattern 373 ==============
============== Pattern 374 ==============
============== Pattern 375 ==============
============== Pattern 376 ==============
============== Pattern 377 ==============
============== Pattern 378 ==============
============== Pattern 379 ==============
============== Pattern 380 ==============
============== Pattern 381 ==============
============== Pattern 382 ==============
============== Pattern 383 ==============
============== Pattern 384 ==============
============== Pattern 385 ==============
============== Pattern 386 ==============
============== Pattern 387 ==============
============== Pattern 388 ==============
============== Pattern 389 ==============
============== Pattern 390 ==============
============== Pattern 391 ==============
============== Pattern 392 ==============
============== Pattern 393 ==============
============== Pattern 394 ==============
============== Pattern 395 ==============
============== Pattern 396 =======

============== Pattern 1469 ==============
============== Pattern 1470 ==============
============== Pattern 1471 ==============
============== Pattern 1472 ==============
============== Pattern 1473 ==============
============== Pattern 1474 ==============
============== Pattern 1475 ==============
============== Pattern 1476 ==============
============== Pattern 1477 ==============
============== Pattern 1478 ==============
============== Pattern 1479 ==============
============== Pattern 1480 ==============
============== Pattern 1481 ==============
============== Pattern 1482 ==============
============== Pattern 1483 ==============
============== Pattern 1484 ==============
============== Pattern 1485 ==============
============== Pattern 1486 ==============
============== Pattern 1487 ==============
============== Pattern 1488 ==============
============== Pattern 1489 ==============
============== Pattern 1490 ==============
============== Pattern 1491 ==============
===========

============== Pattern 1730 ==============
============== Pattern 1731 ==============
============== Pattern 1732 ==============
============== Pattern 1733 ==============
============== Pattern 1734 ==============
============== Pattern 1735 ==============
============== Pattern 1736 ==============
============== Pattern 1737 ==============
============== Pattern 1738 ==============
============== Pattern 1739 ==============
============== Pattern 1740 ==============
============== Pattern 1741 ==============
============== Pattern 1742 ==============
============== Pattern 1743 ==============
============== Pattern 1744 ==============
============== Pattern 1745 ==============
============== Pattern 1746 ==============
============== Pattern 1747 ==============
============== Pattern 1748 ==============
============== Pattern 1749 ==============
============== Pattern 1750 ==============
============== Pattern 1751 ==============
============== Pattern 1752 ==============
===========

============== Pattern 2467 ==============
============== Pattern 2468 ==============
============== Pattern 2469 ==============
============== Pattern 2470 ==============
============== Pattern 2471 ==============
============== Pattern 2472 ==============
============== Pattern 2473 ==============
============== Pattern 2474 ==============
============== Pattern 2475 ==============
============== Pattern 2476 ==============
============== Pattern 2477 ==============
============== Pattern 2478 ==============
============== Pattern 2479 ==============
============== Pattern 2480 ==============
============== Pattern 2481 ==============
============== Pattern 2482 ==============
============== Pattern 2483 ==============
============== Pattern 2484 ==============
============== Pattern 2485 ==============
============== Pattern 2486 ==============
============== Pattern 2487 ==============
============== Pattern 2488 ==============
============== Pattern 2489 ==============
===========

============== Pattern 2727 ==============
============== Pattern 2728 ==============
============== Pattern 2729 ==============
============== Pattern 2730 ==============
============== Pattern 2731 ==============
============== Pattern 2732 ==============
============== Pattern 2733 ==============
============== Pattern 2734 ==============
============== Pattern 2735 ==============
============== Pattern 2736 ==============
============== Pattern 2737 ==============
============== Pattern 2738 ==============
============== Pattern 2739 ==============
============== Pattern 2740 ==============
============== Pattern 2741 ==============
============== Pattern 2742 ==============
============== Pattern 2743 ==============
============== Pattern 2744 ==============
============== Pattern 2745 ==============
============== Pattern 2746 ==============
============== Pattern 2747 ==============
============== Pattern 2748 ==============
============== Pattern 2749 ==============
===========

============== Pattern 3006 ==============
============== Pattern 3007 ==============
============== Pattern 3008 ==============
============== Pattern 3009 ==============
============== Pattern 3010 ==============
============== Pattern 3011 ==============
============== Pattern 3012 ==============
============== Pattern 3013 ==============
============== Pattern 3014 ==============
============== Pattern 3015 ==============
============== Pattern 3016 ==============
============== Pattern 3017 ==============
============== Pattern 3018 ==============
============== Pattern 3019 ==============
============== Pattern 3020 ==============
============== Pattern 3021 ==============
============== Pattern 3022 ==============
============== Pattern 3023 ==============
============== Pattern 3024 ==============
============== Pattern 3025 ==============
============== Pattern 3026 ==============
============== Pattern 3027 ==============
============== Pattern 3028 ==============
===========

============== Pattern 3287 ==============
============== Pattern 3288 ==============
============== Pattern 3289 ==============
============== Pattern 3290 ==============
============== Pattern 3291 ==============
============== Pattern 3292 ==============
============== Pattern 3293 ==============
============== Pattern 3294 ==============
============== Pattern 3295 ==============
============== Pattern 3296 ==============
============== Pattern 3297 ==============
============== Pattern 3298 ==============
============== Pattern 3299 ==============
============== Pattern 3300 ==============
============== Pattern 3301 ==============
============== Pattern 3302 ==============
============== Pattern 3303 ==============
============== Pattern 3304 ==============
============== Pattern 3305 ==============
============== Pattern 3306 ==============
============== Pattern 3307 ==============
============== Pattern 3308 ==============
============== Pattern 3309 ==============
===========

============== Pattern 3552 ==============
============== Pattern 3553 ==============
============== Pattern 3554 ==============
============== Pattern 3555 ==============
============== Pattern 3556 ==============
============== Pattern 3557 ==============
============== Pattern 3558 ==============
============== Pattern 3559 ==============
============== Pattern 3560 ==============
============== Pattern 3561 ==============
============== Pattern 3562 ==============
============== Pattern 3563 ==============
============== Pattern 3564 ==============
============== Pattern 3565 ==============
============== Pattern 3566 ==============
============== Pattern 3567 ==============
============== Pattern 3568 ==============
============== Pattern 3569 ==============
============== Pattern 3570 ==============
============== Pattern 3571 ==============
============== Pattern 3572 ==============
============== Pattern 3573 ==============
============== Pattern 3574 ==============
===========